In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict
import numpy as np
from datetime import datetime, timedelta

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 3
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2023年04月14日"  # include
END_DATE = "2023年04月27日"  # include

WEEKS = [(START_DATE, END_DATE)]
current_date = START_DATE
while current_date < END_DATE:
    current_date_obj = datetime.strptime(current_date, '%Y年%m月%d日')
    next_date_obj = current_date_obj + timedelta(days=6)
    next_date = next_date_obj.strftime('%Y年%m月%d日')
    if next_date > END_DATE:
        next_date = END_DATE
    WEEKS.append((current_date, next_date))

    current_date_obj = next_date_obj + timedelta(days=1)
    current_date = current_date_obj.strftime('%Y年%m月%d日')
print(WEEKS)

TARGET_CATEGORY = "起源帕路奇亞VSTAR"

"""
統計 and 資料分析
古劍豹ex
古鼎鹿ex
古簡蝸ex

放逐_騎拉帝納VSTAR

沙奈朵ex
沙奈朵_飄飄球
沙奈朵_蒼響

LTB
LTB_勾魂眼_噴火龍
LTB_蓋歐卡
LTB_洗翠黏美龍VSTAR

阿爾宙斯VSTAR
阿爾_騎拉帝納VSTAR
阿爾_阿羅拉六尾VSTAR

夢幻VMAX

密勒頓ex

洛奇亞VSTAR
一擊_洛奇亞
純白_洛奇亞

- 起源帕路奇亞VSTAR

--
統計 only

雷吉鐸拉戈VSTAR

柱神

飄香豚ex

團結之翼



"""

[('2023年04月14日', '2023年04月27日'), ('2023年04月14日', '2023年04月20日'), ('2023年04月21日', '2023年04月27日')]


'\n統計 and 資料分析\n古劍豹ex\n古鼎鹿ex\n古簡蝸ex\n\n放逐_騎拉帝納VSTAR\n\n沙奈朵ex\n沙奈朵_飄飄球\n沙奈朵_蒼響\n\nLTB\nLTB_勾魂眼_噴火龍\nLTB_蓋歐卡\nLTB_洗翠黏美龍VSTAR\n\n阿爾宙斯VSTAR\n阿爾_騎拉帝納VSTAR\n阿爾_阿羅拉六尾VSTAR\n\n夢幻VMAX\n\n密勒頓ex\n\n洛奇亞VSTAR\n一擊_洛奇亞\n純白_洛奇亞\n\n- 起源帕路奇亞VSTAR\n\n--\n統計 only\n\n雷吉鐸拉戈VSTAR\n\n柱神\n\n飄香豚ex\n\n團結之翼\n\n\n\n'

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['交錯切換機', 'LTB', 'LTB_勾魂眼_噴火龍', '光輝無極汰那', '古劍豹ex', '夢幻VMAX', '起源帕路奇亞VSTAR', '密勒頓ex', '密勒頓ex_other', '白馬蕾冠王VMAX', '達克萊伊VSTAR', 'LTB_蓋歐卡', 'LTB_快龍V', 'LTB_空之封印石_other', 'LTB_空之封印石_悪水超雷', 'LTB_洗翠黏美龍VSTAR', 'LTB_空之封印石_水超闘雷', 'LTB_空之封印石_水超雷', 'LTB_空之封印石_水超鋼闘雷', 'LTB_空之封印石_水超鋼雷', '魔幻假面喵ex', 'LTB_空之封印石_水草超闘雷', 'LTB_空之封印石_水超闘', 'LTB_空之封印石_悪水超', 'LTB_空之封印石_水超鋼', '黑馬蕾冠王VMAX', '連擊武道熊師VMAX', '阿羅拉六尾VSTAR', '酋雷姆VMAX', '雷吉艾勒奇VMAX', '飛翔皮卡丘VMAX', '密勒頓ex_電柱', '密勒頓ex_電柱_茸茸羊', '密勒頓ex_茸茸羊', 'Other_Lost', '洛托姆VSTAR', '阿爾宙斯VSTAR', '阿爾_白馬VMAX', '純阿爾VSTAR', '洛奇亞VSTAR', '連擊_洛奇亞', '阿爾_飛天皮VMAX', '阿爾_阿羅拉六尾VSTAR', '洗翠索羅亞克VSTAR', 'Other_騎拉帝納VSTAR', '阿爾_騎拉帝納VSTAR', '阿爾_鋁鋼龍VMAX', '雷吉鐸拉戈VSTAR', '阿爾_雷吉鐸拉戈VSTAR', '藏飽栗鼠VMAX', '烈空坐VMAX', '化石翼龍VSTAR', '起源帝牙盧卡VSTAR', '古簡蝸ex', '純白_洛奇亞', '一擊_洛奇亞', '古鼎鹿ex', '月石', '沙奈朵ex', '沙奈朵_飄飄球', '沙奈朵_蒼響', '超夢 V-UNION', '沙奈朵_蒼響_超夢V-UNION', '沙奈朵_超夢V-UNION', '放逐_騎拉帝納VSTAR', '音波龍ex', 'others', '團結之翼', '飄香豚ex', '柱神', '幻影索羅亞克', '火焰雞VMAX', '索羅風速狗'])


numbe

In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


7.414817810058594e-05 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# # show all categories we have
# total = 0
# for k in decks.keys():
#     print(f"[{k}]:\n{len(decks[k])}\n")
#     total += len(decks[k])

# print(total)

In [10]:
# show categories we have in the period
real_start_date, real_end_date = "3000年01月01日", "1000年01月01日"
number_target_deck = 0
deck_cnt_list = []
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str >= START_DATE and date_exclude_jp_str <= END_DATE:
            deck_cnt += 1

            if date_exclude_jp_str < real_start_date:
                real_start_date = date_exclude_jp_str

            if date_exclude_jp_str > real_end_date:
                real_end_date = date_exclude_jp_str

    # print(f"{k}\t{deck_cnt}\n")
    deck_cnt_list.append((deck_cnt, k))
    if k == TARGET_CATEGORY:
        number_target_deck = deck_cnt

In [11]:
# # Show all categories with counter
# deck_cnt_list = sorted(deck_cnt_list, reverse=True)
# for num_deck, deck_name in deck_cnt_list:
#     if num_deck > 0:
#         print(f"{deck_name}\t{num_deck}\n")

In [12]:
# Show number of decks for the target category
print(number_target_deck)
if number_target_deck <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

53


In [13]:
def fill_na(col):
    if col.name not in ["date", "prefecture"]:
        return col.fillna(0)
    else:
        return col

In [14]:
# Analysis
df_dict = {}
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        if deck["date"] < START_DATE or deck["date"] > END_DATE:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.apply(fill_na)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # Calculate for each week
    for week_id, week in enumerate(WEEKS):
        week_name = f"week_{week_id}" if week_id > 0 else "all"
        week_start, week_end = week
        df_week = df.loc[(df['date'] >= week_start) & (df['date'] <= week_end)]

        # calculate
        num_decks = df_week.shape[0]
        num_used = df_week[col_list].sum(axis="rows", numeric_only=True)
        num_picked = df_week[col_list].astype(bool).sum(axis="rows")
        avg_num_used = num_used / num_picked
        pick_rate = num_picked / num_decks

        # insert rows in df
        avg_idx = f"avg_num_used_{week_name}\n{week[0]}\n{week[1]}"
        pick_idx = f"pick_rate_{week_name}\n{week[0]}\n{week[1]}"
        df.loc[avg_idx] = {}
        df.loc[pick_idx] = {}
        for col in col_list:
            df.loc[avg_idx, col] = avg_num_used[col] if not np.isnan(avg_num_used[col]) else 0
            df.loc[pick_idx, col] = pick_rate[col] if not np.isnan(pick_rate[col]) else 0

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    num_weeks = len(WEEKS)
    target_rows = [num_rows - i for i in range(num_weeks*2-1, 0, -2)]
    target_rows += [num_rows - i for i in range(num_weeks*2, 1, -2)]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort by card name which is header of df
    sorted_cols = np.argsort(df.iloc[0, 4:])
    sorted_cols = list(sorted_cols.to_dict().keys())
    df = df[common_cols + sorted_cols]

    # sort cols by pick rate
    df = df.sort_values(f"pick_rate_all\n{WEEKS[0][0]}\n{WEEKS[0][1]}", axis=1, ascending=False)
    
    # reorder header, move common cols to front
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_dict[card_type] = df

In [15]:
# Detect repeated card names
from collections import Counter
card_full_names = df_dict["pokemons"].columns[4:].tolist()
card_ch_names = [full_name.split("\n")[0] for full_name in card_full_names]
card_name_dict = {ch_name: [] for ch_name in card_ch_names}
for full_name in card_full_names:
    ch_name = full_name.split("\n")[0]
    card_name_dict[ch_name].append(full_name)

for ch_name, full_name_list in card_name_dict.items():
    if len(full_name_list) > 1:
        print(ch_name + ":")
        for full_name in full_name_list:
            print("- " + full_name)
        print('======')

涼脊龍:
- 涼脊龍
セビエ
SV2P 024/071
- 涼脊龍
セビエ
SV2P 025/071


In [16]:
# Detect if card name is not translated yet
import unicodedata

# Define a function to check if a string is combination of Chinese, English and Number
def is_ch_en_num(string):
    for char in string:
        # Check if the character is not in the Chinese, English and Number
        uni_data = unicodedata.name(char, '').split(" ")
        if 'CJK' not in uni_data and \
            'LATIN' not in uni_data and \
            'DIGIT' not in uni_data and \
            uni_data[-1] not in ["DOT", "SPACE", "STOP"]:
            return False
    return True

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    card_full_names = df_dict[card_type].columns[4:].tolist()
    for name in card_full_names:
        card_ch_names = name.split("\n")[0]
        if not is_ch_en_num(card_ch_names): print(name)

In [17]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{START_DATE}-{END_DATE}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_dict[sheet_name].to_excel(writer, sheet_name=sheet_name, float_format="%.3f")

    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]

    # Make deck_link become a hyperlink
    for i, deck_id in enumerate(df_dict[sheet_name].index):
        if "avg_num_used" in deck_id or "pick_rate" in deck_id:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    header_format.set_bottom(2)
    for col_num, value in enumerate(df_dict[sheet_name].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)

    format_bold_bottom = workbook.add_format()
    format_bold_bottom.set_bottom(2)
    rows_for_bold = [0 + len(WEEKS), 0 + len(WEEKS)*2]
    for row in rows_for_bold:
        worksheet.set_row(row, None, format_bold_bottom)
    worksheet.autofit()

workbook.close()
writer.save()

/Users/blake/anaconda3/envs/deck/lib/python3.10/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [18]:
# Show decks have no category
other_decks = decks["others"]
other_decks = sorted(other_decks, key=lambda deck: deck["date"], reverse=True)
print(len(other_decks))
for d in other_decks:
    date = d["date"]
    link = d["deck_link"]
    rank = d["rank"]
    num_p = d["num_players"]
    print(f"{date}\t{rank}/{num_p}\t{link}")

16
2023年04月25日(火)	12/64	https://www.pokemon-card.com/deck/confirm.html/deckID/48x48a-9gKzSa-8cDx84
2023年04月23日(日)	5/64	https://www.pokemon-card.com/deck/confirm.html/deckID/F5Vkkk-YRO0Qt-fFkvV5
2023年04月23日(日)	5/64	https://www.pokemon-card.com/deck/confirm.html/deckID/5F5kdk-p4NnVr-fkFFdf
2023年04月23日(日)	4/64	https://www.pokemon-card.com/deck/confirm.html/deckID/c8xc48-FiX6uN-GacDD4
2023年04月22日(土)	9/48	https://www.pokemon-card.com/deck/confirm.html/deckID/Dx8x88-Qx7FYR-488x8c
2023年04月20日(木)	5/70	https://www.pokemon-card.com/deck/confirm.html/deckID/HHigQL-jBAyH4-gNLngn
2023年04月17日(月)	6/32	https://www.pokemon-card.com/deck/confirm.html/deckID/fFFk5F-wFAvDR-fbdVFV
2023年04月16日(日)	2/32	https://www.pokemon-card.com/deck/confirm.html/deckID/Vkkffk-BqjoNF-5f5vdF
2023年04月16日(日)	5/32	https://www.pokemon-card.com/deck/confirm.html/deckID/nLnnQQ-SkQ495-HNHggH
2023年04月16日(日)	11/48	https://www.pokemon-card.com/deck/confirm.html/deckID/XEMp22-po8KnV-XRpMyy
2023年04月15日(土)	7/64	https://www.pokemon-card.

In [19]:
#!cp -r ~/Projects/top_pokecard_deck_analysis/scripts/excel/*  ~/Google\ Drive/我的雲端硬碟/deck_analysis/excel/

In [20]:
# Sync files to google drive
import shutil

# Define source and destination directories
src_dir = f'/Users/blake/Projects/top_pokecard_deck_analysis/scripts/excel/{TARGET_CATEGORY}'
dst_dir = f'/Users/blake/Google Drive/我的雲端硬碟/deck_analysis/excel/{TARGET_CATEGORY}'

# Use os.listdir() to get a list of all files in the source directory
files = os.listdir(src_dir)

# if dst_dir not exists, create one
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Use a for loop to copy each file to the destination directory
for file in files:
    # Define the full path of the source file
    src_file = os.path.join(src_dir, file)
    # Define the full path of the destination file
    dst_file = os.path.join(dst_dir, file)
    # Use shutil.copy() to copy the source file to the destination directory
    ret = shutil.copy(src_file, dst_file)

    print(ret)

/Users/blake/Google Drive/我的雲端硬碟/deck_analysis/excel/起源帕路奇亞VSTAR/起源帕路奇亞VSTAR-2023年04月14日-2023年04月27日.xlsx
/Users/blake/Google Drive/我的雲端硬碟/deck_analysis/excel/起源帕路奇亞VSTAR/起源帕路奇亞VSTAR-2023年04月14日-2023年04月20日.xlsx
